In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm, trange
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import pymorphy2
from pymystem3 import Mystem as mystem
dt = pd.read_excel("/Реестр 327 тыс. деклараций ЕП РФ без 140000-200000.xlsx", 0)
dt.columns = [x.replace(" ", "_") for x in dt.columns]
dt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,Общее_наименование_продукции,Раздел_ЕП_РФ_(Код_из_ФГИС_ФСА_для_подкатегории_продукции),Подкатегория_продукции
0,1,"Парацетамол таблетки 500 мг 10 шт., упаковки я...",9300.1,"Лекарственные средства, зарегистрированные в у..."
1,2,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...
2,3,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...
3,4,Аппарат вакуумно-лазерной терапии стоматологич...,9444.4,Приборы и аппараты для электролечения высокоча...
4,5,Блоки оконные и балконные дверные из алюминиев...,5270.1,Блоки оконные и балконные дверные из алюминиев...
...,...,...,...,...
267679,327680,"Выключатели вакуумные серии ВР1, на номинально...",3414.8,Выключатели силовые
267680,327681,Виниловые обои на флизелиновой основе,5460.1,Обои
267681,327682,Смеси бетонные,2364.1,Смеси строительные
267682,327683,Кормовая добавка для животных,9296.1,Комбикорма и добавки белково-витаминные для не...


In [ ]:
def razb(txt, sp):
  for i in range(txt.count(";")):
    id = txt.index(";")
    sp.append(txt[:id])
    txt = txt[id+2:]
  sp.append(txt)
  return sp

dop_sp = []
delet = []
n, m = dt.shape
for i in range(n):
  if ";" in dt.iloc[i, 2]:
    delet.append(i)
    nu = []
    st = []
    razb(dt.iloc[i, 2], nu)
    razb(dt.iloc[i, 3], st)
    for j in range(len(nu)):
        dop_sp.append([dt.iloc[i, 0], dt.iloc[i, 1], nu[j], st[j]])
for k in dop_sp:
  dt.loc[len(dt.index)] = k
for el in delet[::-1]:
  dt = dt.drop(el)

In [ ]:
dt.iloc[10:20]

,id,Общее_наименование_продукции,Раздел_ЕП_РФ_(Код_из_ФГИС_ФСА_для_подкатегории_продукции),Подкатегория_продукции
10,11,"лекарственное средство: ФУРАЦИЛИН, таблетки дл...",9300.1,"Лекарственные средства, зарегистрированные в у..."
11,12,"лекарственное средство: ФУРАЦИЛИН, таблетки дл...",9300.1,"Лекарственные средства, зарегистрированные в у..."
12,13,"лекарственное средство: ФУРАЦИЛИН, таблетки дл...",9300.1,"Лекарственные средства, зарегистрированные в у..."
13,14,"лекарственное средство: ФУРАЦИЛИН, таблетки дл...",9300.1,"Лекарственные средства, зарегистрированные в у..."
14,15,"лекарственное средство: ФУРАЦИЛИН, таблетки дл...",9300.1,"Лекарственные средства, зарегистрированные в у..."
15,16,"лекарственное средство: ФУРАЦИЛИН, таблетки дл...",9300.1,"Лекарственные средства, зарегистрированные в у..."
16,17,Лекарственный препарат Вода для инъекций буфус...,9300.1,"Лекарственные средства, зарегистрированные в у..."
17,18,салфетки бумажные сервировочные с маркировкой ...,5463.1,Изделия из бумаги бытового и санитарно-гигиени...
21,22,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...
23,24,лекарственное средство: Ацетилсалициловая кисл...,9300.1,"Лекарственные средства, зарегистрированные в у..."


In [ ]:
for i in range(n):
  if ";" in dt.iloc[i, 2]:
    print("!" + str(i))

In [ ]:
dt_sort = dt.sort_values("Подкатегория_продукции")
dt_sort.head()

,id,Общее_наименование_продукции,Раздел_ЕП_РФ_(Код_из_ФГИС_ФСА_для_подкатегории_продукции),Подкатегория_продукции
43516,43517,Инвентарь для прикладных видов спорта: Луки сп...,9616.2,Холодное оружие: луки спортивные массовые
246307,306308,конструктивно сходное с холодным клинковым ору...,9616.3,Холодное оружие: сабли
252059,312060,Посуда из коррозионно-стойкой стали для взросл...,9616.4,Холодное оружие: шпаги
224894,284895,"Вакуумный выключатель,",9683.1,Автоматические устройства управления бытовыми ...
244207,304208,Устройства вводно-распределительные для жилых ...,9683.1,Автоматические устройства управления бытовыми ...


Предобработка текста

In [ ]:
import string
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
mystem = Mystem() 

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...'])
def lemmatize_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)
    return text

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text))))
dt_sort['preproccessed'] = list(map(preproccessing, dt_sort["Общее_наименование_продукции"]))
dt_sort.head()

,id,Общее_наименование_продукции,Раздел_ЕП_РФ_(Код_из_ФГИС_ФСА_для_подкатегории_продукции),Подкатегория_продукции,preproccessed
43516,43517,Инвентарь для прикладных видов спорта: Луки сп...,9616.2,Холодное оружие: луки спортивные массовые,Инвентарь для прикладных видов спорта Луки спо...
246307,306308,конструктивно сходное с холодным клинковым ору...,9616.3,Холодное оружие: сабли,конструктивно сходное с холодным клинковым ору...
252059,312060,Посуда из коррозионно-стойкой стали для взросл...,9616.4,Холодное оружие: шпаги,Посуда из коррозионно стойкой стали для взросл...
224894,284895,"Вакуумный выключатель,",9683.1,Автоматические устройства управления бытовыми ...,Вакуумный выключатель
244207,304208,Устройства вводно-распределительные для жилых ...,9683.1,Автоматические устройства управления бытовыми ...,Устройства вводно распределительные для жилых ...


In [ ]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(dt_sort["Общее_наименование_продукции"])]

  0%|          | 0/268914 [00:00<?, ?it/s]

In [ ]:
dt_sort['text_prep'] = prep_text

Стемминг

In [ ]:
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer("russian") 

In [ ]:
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])

In [ ]:
from nltk import word_tokenize
text = dt_sort["text_prep"][0]
word_tokenize(text)

['парацетамол',
 'таблетки',
 'мг',
 'шт',
 'упаковки',
 'ячейковые',
 'контурные',
 'пачки',
 'картонные',
 'рег',
 '№',
 'лс',
 'от',
 'серия',
 'партия',
 'упаковок',
 'годен',
 'до',
 'производства',
 'оао',
 'фармстандарт',
 'лексредства',
 'инн',
 'курская',
 'область',
 'курск',
 'ул',
 'я',
 'агрегатная',
 'а',
 'россия',
 'код',
 'окпд']

In [ ]:
from nltk import word_tokenize

stemmed_texts_list = []
for text in tqdm(dt_sort['text_prep']):
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

dt_sort['text_stem'] = stemmed_texts_list

  0%|          | 0/268914 [00:00<?, ?it/s]

In [ ]:
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

In [ ]:
from nltk import word_tokenize

sw_texts_list = []
for text in tqdm(dt_sort['text_prep']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

dt_sort ['text_sw'] = sw_texts_list

  0%|          | 0/268914 [00:00<?, ?it/s]

In [ ]:
dt_sort.iloc[:, 4:]


Леммитизация

In [ ]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

lem_txt = []
for txt in tqdm(dt_sort['text_sw']):
  x = lemmatize(txt)
  l = " ".join(x)  
  lem_txt.append(l)
dt_sort['text_lem'] = lem_txt

  0%|          | 0/268914 [00:00<?, ?it/s]

In [ ]:
dt_sort

,id,Общее_наименование_продукции,Раздел_ЕП_РФ_(Код_из_ФГИС_ФСА_для_подкатегории_продукции),Подкатегория_продукции,preproccessed,text_prep,text_stem,text_sw,text_lem
43516,43517,Инвентарь для прикладных видов спорта: Луки сп...,9616.2,Холодное оружие: луки спортивные массовые,Инвентарь для прикладных видов спорта Луки спо...,инвентарь для прикладных видов спорта луки спо...,инвентар прикладн вид спорт лук спортивн массо...,инвентарь прикладных видов спорта луки спортив...,инвентарь прикладной вид спорт лука спортивный...
246307,306308,конструктивно сходное с холодным клинковым ору...,9616.3,Холодное оружие: сабли,конструктивно сходное с холодным клинковым ору...,конструктивно сходное с холодным клинковым ору...,конструктивн сходн холодн клинков оруж сувенир...,конструктивно сходное холодным клинковым оружи...,конструктивно сходный холодный клинковый оружи...
252059,312060,Посуда из коррозионно-стойкой стали для взросл...,9616.4,Холодное оружие: шпаги,Посуда из коррозионно стойкой стали для взросл...,посуда из коррозионно стойкой стали для взросл...,посуд коррозион стойк стал взросл кастрюл котл...,посуда коррозионно стойкой стали взрослых каст...,посуда коррозионный стойка стать взрослый каст...
224894,284895,"Вакуумный выключатель,",9683.1,Автоматические устройства управления бытовыми ...,Вакуумный выключатель,вакуумный выключатель,вакуумн выключател,вакуумный выключатель,вакуумный выключатель
244207,304208,Устройства вводно-распределительные для жилых ...,9683.1,Автоматические устройства управления бытовыми ...,Устройства вводно распределительные для жилых ...,устройства вводно распределительные для жилых ...,устройств вводн распределительн жил обществен ...,устройства вводно распределительные жилых обще...,устройство вводный распределительный жилой общ...
...,...,...,...,...,...,...,...,...,...
225044,285045,Бинт гипсовый медицинский под товарным знаком ...,9393.2,бинты гипсовые медицинские,Бинт гипсовый медицинский под товарным знаком ...,бинт гипсовый медицинский под товарным знаком ...,бинт гипсов медицинск товарн знак matopat вари...,бинт гипсовый медицинский товарным знаком mato...,бинт гипсовый медицинский товарный знак matopa...
227076,287077,Бинт гипсовый быстросхватывающийся четырехслой...,9393.2,бинты гипсовые медицинские,Бинт гипсовый быстросхватывающийся четырехслой...,бинт гипсовый быстросхватывающийся четырехслой...,бинт гипсов быстросхватыва четырехслойн gipset...,бинт гипсовый быстросхватывающийся четырехслой...,бинт гипсовый быстросхватывающийся четырехслой...
203995,263996,"Бинты медицинские трикотажные, трубчатые, под ...",9393.2,бинты гипсовые медицинские,Бинты медицинские трикотажные трубчатые под ги...,бинты медицинские трикотажные трубчатые под ги...,бинт медицинск трикотажн трубчат гипс доктор бинт,бинты медицинские трикотажные трубчатые гипс д...,бинт медицинский трикотажный трубчатый гипс до...
101885,101886,"Гипс медицинский, марки: Г-5, Г-6, Г-16, Г-18",9393.2,бинты гипсовые медицинские,Гипс медицинский марки Г Г Г Г,гипс медицинский марки г г г г,гипс медицинск марк г г г г,гипс медицинский марки г г г г,гипс медицинский марка г г г г


In [ ]:
rez = []
for txt in tqdm(dt_sort['Подкатегория_продукции'].unique()):
  if ";" in txt:
    for i in range(txt.count(";")):
      id = txt.index(";")
      rez.append(txt[:id])
      txt = txt[id+2:]
  rez.append(txt)
my_tags = list(set(rez))
print(my_tags)
y = my_tags

  0%|          | 0/386 [00:00<?, ?it/s]

['Продукция мукомольно-крупяной промышленности кормовая', 'Инструменты механизированные, инструменты сшивающие', 'Материалы вспомогательные', 'Кабели силовые для стационарной прокладки на напряжение свыше 1 кВ', 'Комбикорма для пушных зверей, кроликов и нутрий', 'Корма животного происхождения (включая корма для непродуктивных животных)', 'Материалы хирургические, средства перевязочные: изделия ватно-марлевые медицинские', 'Аккумуляторы и аккумуляторные батареи никель-металлгидридной и литиевой систем', 'Сифоны бытовые и баллончики к ним', 'Рыбий жир (кроме жира технического)', 'Изделия протезно-ортопедические:', 'Пленки радиографические медицинские', 'Кормовая продукция маслобойной и жировой промышленности (жмыхи и шроты)', 'Тест-системы для диагностики других инфекционных заболеваний', 'Материалы теплоизоляционные из пеностекла', 'Оборудование медицинское', 'Приборы и аппараты для лечения, инкубаторы детские', 'Измерительные установки, комплексы, сигнализаторы, регистраторы, мониторы 

In [ ]:
my_tags

['Продукция мукомольно-крупяной промышленности кормовая',
 'Инструменты механизированные, инструменты сшивающие',
 'Материалы вспомогательные',
 'Кабели силовые для стационарной прокладки на напряжение свыше 1 кВ',
 'Комбикорма для пушных зверей, кроликов и нутрий',
 'Корма животного происхождения (включая корма для непродуктивных животных)',
 'Материалы хирургические, средства перевязочные: изделия ватно-марлевые медицинские',
 'Аккумуляторы и аккумуляторные батареи никель-металлгидридной и литиевой систем',
 'Сифоны бытовые и баллончики к ним',
 'Рыбий жир (кроме жира технического)',
 'Изделия протезно-ортопедические:',
 'Пленки радиографические медицинские',
 'Кормовая продукция маслобойной и жировой промышленности (жмыхи и шроты)',
 'Тест-системы для диагностики других инфекционных заболеваний',
 'Материалы теплоизоляционные из пеностекла',
 'Оборудование медицинское',
 'Приборы и аппараты для лечения, инкубаторы детские',
 'Измерительные установки, комплексы, сигнализаторы, регист

In [ ]:
X = dt_sort['text_lem']
y = dt_sort['Подкатегория_продукции']


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)
print(X_test)

40108     лекарственный средство натрий хлорид солофарма...
73186     коронал таблетка покрыть плёночный оболочка мг...
255893    посуда коррозионностойкий стать элемент пластм...
175412    остеогенон таблетка покрыть оболочка мг шт бли...
222737    посуда алюминиевый литой противопригарать покр...
                                ...                        
68922     глицин таблетка подъязычный мг шт упаковка яче...
249999                                    жмых подсолнечный
10143                     лекарственный средство мерифатина
40396     энтеросгель® паста приём внутрь г туба комбини...
111715    лефлобакт® раствор инфузия мг мл мл флакон тём...
Name: text_lem, Length: 80675, dtype: object


Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [ ]:
%%time
nb.fit(X_train, y_train)

CPU times: user 8.26 s, sys: 1.15 s, total: 9.41 s
Wall time: 9.42 s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [ ]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

CPU times: user 2.4 s, sys: 19.9 ms, total: 2.42 s
Wall time: 2.43 s


In [ ]:
y_pred[0]

'Корма животного происхождения (включая корма для непродуктивных животных)'

In [ ]:
print(X_test[0], y_test[0], y_pred[0])

парацетамол таблетка мг шт упаковка ячейковый контурный пачка картонный рег № лс серия партия упаковка годный производство оао фармстандарт лексредство инна курский область курск ул агрегатный россия код окпд Лекарственные средства, зарегистрированные в установленном порядке и внесенные в государственный реестр, состоящие из смешанных и несмешанных продуктов для использования в терапевтических или профилактических целях, расфасованные в виде дозированных лекарственных форм или в упаковки для розничной продажи Корма животного происхождения (включая корма для непродуктивных животных)


In [ ]:
from sklearn.metrics import accuracy_score

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

Linear Support Vector Machine

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [ ]:
%%time
sgd.fit(X_train, y_train)

CPU times: user 1min 49s, sys: 37.1 s, total: 2min 26s
Wall time: 1min 42s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [ ]:
%%time
y_pred = sgd.predict(X_test)

CPU times: user 2.04 s, sys: 33.8 ms, total: 2.08 s
Wall time: 2.08 s


In [ ]:
x = "Лекарственные средства, зарегистрированные в установленном порядке и внесенные в государственный реестр, состоящие из смешанных и несмешанных продуктов для использования в терапевтических или профилактических целях, расфасованные в виде дозированных лекарственных форм или в упаковки для розничной продажи"

In [ ]:
from sklearn.metrics import classification_report

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))
table = classification_report(y_test, y_pred)

#for line in table:
 # print(line)
 # if re.search(x, line):
    #print(line)
  #  print(1)

In [ ]:
nwt = pd.read_excel("/Реестр 327 тыс. деклараций ЕП РФ без 140000-200000.xlsx", 0)

In [ ]:
nwt

,id,Общее наименование продукции,Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции),Подкатегория продукции
0,1,"Парацетамол таблетки 500 мг 10 шт., упаковки я...",9300.1,"Лекарственные средства, зарегистрированные в у..."
1,2,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...
2,3,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...
3,4,Аппарат вакуумно-лазерной терапии стоматологич...,9444.4,Приборы и аппараты для электролечения высокоча...
4,5,Блоки оконные и балконные дверные из алюминиев...,5270.1,Блоки оконные и балконные дверные из алюминиев...
...,...,...,...,...
267679,327680,"Выключатели вакуумные серии ВР1, на номинально...",3414.8,Выключатели силовые
267680,327681,Виниловые обои на флизелиновой основе,5460.1,Обои
267681,327682,Смеси бетонные,2364.1,Смеси строительные
267682,327683,Кормовая добавка для животных,9296.1,Комбикорма и добавки белково-витаминные для не...


In [ ]:
def razb(txt, sp):
  for i in range(txt.count(";")):
    id = txt.index(";")
    sp.append(txt[:id])
    txt = txt[id+2:]
  sp.append(txt)
  return sp

dop_sp = []
delet = []
n, m = nwt.shape
for i in range(n):
  if ";" in nwt.iloc[i, 2]:
    delet.append(i)
    nu = []
    st = []
    razb(nwt.iloc[i, 2], nu)
    razb(nwt.iloc[i, 3], st)
    for j in range(len(nu)):
        dop_sp.append([nwt.iloc[i, 0], nwt.iloc[i, 1], nu[j], st[j]])
for k in dop_sp:
  nwt.loc[len(nwt.index)] = k
for el in delet[::-1]:
  nwt = nwt.drop(el)

In [ ]:
nwt

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
logreg = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [ ]:
dbm = []
bkb = table.split('\n')
for i in range(len(bkb)- 1):
  dbm.append(bkb[i + 1].strip())
  print(bkb[i + 1].strip())

In [ ]:
def progrm(x):
  for txt in dbm:
    fk = txt.split('       ')
    if str(fk[0]) == str(x):
      return int(float(fk[1][:4]) * 100)

In [ ]:
msv = []
for index, row in tqdm(nwt.iterrows()):
  x = str(row['Общее наименование продукции'])
  y = str(row['Подкатегория продукции'])
  ect_pred = sgd.predict([x])
  zn = progrm(y.rstrip())
  try:
    if int(zn) > 75:
      if ect_pred == y:
        msv.append('da')
      else:
        msv.append('niet')
    else:
      msv.append('x3')
  except:
    msv.append('oops')

nwt["Результат"] = msv
  

0it [00:00, ?it/s]

In [ ]:
progrm(x)

94

Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [ ]:
%%time
nb.fit(X_train, y_train)

CPU times: user 8.55 s, sys: 440 ms, total: 8.99 s
Wall time: 9.02 s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [ ]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

CPU times: user 2.35 s, sys: 5.86 ms, total: 2.35 s
Wall time: 2.36 s


In [ ]:
y_pred[0]

'Корма животного происхождения (включая корма для непродуктивных животных)'

In [ ]:
print(X_test[0])
print(y_test[0])
print(y_pred[0])

парацетамол таблетка мг шт упаковка ячейковый контурный пачка картонный рег № лс серия партия упаковка годный производство оао фармстандарт лексредство инна курский область курск ул агрегатный россия код окпд
Лекарственные средства, зарегистрированные в установленном порядке и внесенные в государственный реестр, состоящие из смешанных и несмешанных продуктов для использования в терапевтических или профилактических целях, расфасованные в виде дозированных лекарственных форм или в упаковки для розничной продажи
Корма животного происхождения (включая корма для непродуктивных животных)


In [ ]:
from sklearn.metrics import accuracy_score

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))